<a href="https://colab.research.google.com/github/aryanavar/Fake-News-Classification/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [3]:
import pandas as pd


In [4]:
df=pd.read_csv('/content/drive/MyDrive/train.csv')


In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df=df.dropna()

In [7]:
## Get the Independent Features
X=df.drop('label',axis=1)
## Get the Dependent features
y=df['label']

In [8]:
X.shape
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
### Vocabulary size
voc_size=5000

Onehot representation

In [12]:
messages=X.copy()

In [13]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [14]:
messages.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Processing done by only considering 'title' column from the data

In [17]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [19]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1584, 2031, 2000, 3423, 1651, 3078, 3958, 4846, 3526, 1983],
 [2877, 2542, 742, 3061, 4955, 4095, 3878],
 [373, 3062, 511, 3211],
 [391, 316, 2591, 3117, 4245, 1653],
 [4299, 4955, 1802, 1522, 4102, 4415, 4955, 3669, 4825, 1760],
 [4201,
  137,
  4181,
  227,
  4940,
  2306,
  4435,
  2691,
  309,
  4835,
  2934,
  2613,
  600,
  4173,
  3878],
 [1871, 2375, 3584, 1764, 1298, 2721, 379, 523, 36, 1651, 2033],
 [119, 368, 348, 3128, 1138, 3371, 2306, 851, 36, 1651, 2033],
 [4166, 1944, 2343, 1587, 1694, 293, 3939, 2499, 2306, 169],
 [4032, 2850, 1893, 3421, 4947, 3551, 4315, 1933],
 [1228, 693, 4087, 1585, 341, 4889, 1350, 2290, 2933, 3501, 2959],
 [3117, 1615, 1651, 293, 2306, 1138],
 [4981, 2912, 1025, 355, 4303, 4684, 553, 657, 4809],
 [4932, 2029, 2299, 1237, 4206, 3436, 4082, 36, 1651, 2033],
 [188, 4617, 4507, 4208, 1554, 36, 1651, 2033],
 [3815, 3656, 854, 3400, 3479, 4760, 1014, 3245, 230, 4326],
 [4011, 4707, 2542],
 [3788, 4537, 553, 4446, 2306, 4259, 3843, 3878],
 [2106, 322

### Embedding Representation

In [20]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4846 3526 1983]
 [   0    0    0 ... 4955 4095 3878]
 [   0    0    0 ... 3062  511 3211]
 ...
 [   0    0    0 ...   36 1651 2033]
 [   0    0    0 ... 2846 4505 3901]
 [   0    0    0 ... 3509 4951  565]]


In [21]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1584,
       2031, 2000, 3423, 1651, 3078, 3958, 4846, 3526, 1983], dtype=int32)

In [22]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
len(embedded_docs),y.shape

(18285, (18285,))

In [24]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [27]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 24s 84ms/step - loss: 0.3528 - accuracy: 0.8298 - val_loss: 0.2028 - val_accuracy: 0.9122
Epoch 2/10
192/192 [==============================] - 5s 27ms/step - loss: 0.1509 - accuracy: 0.9403 - val_loss: 0.2063 - val_accuracy: 0.9132
Epoch 3/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1058 - accuracy: 0.9605 - val_loss: 0.1989 - val_accuracy: 0.9188
Epoch 4/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0714 - accuracy: 0.9742 - val_loss: 0.2451 - val_accuracy: 0.9158
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0498 - accuracy: 0.9828 - val_loss: 0.2896 - val_accuracy: 0.9190
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0339 - accuracy: 0.9891 - val_loss: 0.3526 - val_accuracy: 0.9168
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0182 - accuracy: 0.9955 - val_loss: 0.3863 - val_accuracy: 0.9067

Dropout

In [28]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [29]:
y_pred_prob = model.predict(X_test)
y_pred = y_pred_prob.argmax(axis=-1)


189/189 [==============================] - 1s 2ms/step


In [30]:
from sklearn.metrics import confusion_matrix

In [31]:
confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917

Taking the 'text' column into account for processing and modeling part

In [33]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus2 = []
for i in range(0, len(messages)):
    review2 = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review2 = review2.lower()
    review2 = review2.split()
    
    review2 = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review2 = ' '.join(review2)
    corpus.append(review2)

In [34]:
messages['text'][1]

'Ever get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Wellesley, Massachusetts. Everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way Miss Havisham forever wore that wedding dress.  Speaking of Great Expectations, Hillary Rodham overflowed with them 48 years ago when she first addressed a Wellesley graduating class. The president of the college informed those gathered in 1969 that the students needed “no debate so far as I could ascertain as to who their spokesman was to be” (kind of the like the Democratic primaries in 2016 minus the   terms unknown then even at a Seven Sisters school). “I am very glad that Miss Adams made it clear that what I am speaking for today is all of us —  the 400 of us,” Miss Rodham told her classmates. After appointing herself Edger Bergen to the Charlie McCarthys and Mor

In [35]:
corpus2

[]

In [36]:
onehot_repr2=[one_hot(words,voc_size)for words in corpus] 
onehot_repr2

[[1584, 2031, 2000, 3423, 1651, 3078, 3958, 4846, 3526, 1983],
 [2877, 2542, 742, 3061, 4955, 4095, 3878],
 [373, 3062, 511, 3211],
 [391, 316, 2591, 3117, 4245, 1653],
 [4299, 4955, 1802, 1522, 4102, 4415, 4955, 3669, 4825, 1760],
 [4201,
  137,
  4181,
  227,
  4940,
  2306,
  4435,
  2691,
  309,
  4835,
  2934,
  2613,
  600,
  4173,
  3878],
 [1871, 2375, 3584, 1764, 1298, 2721, 379, 523, 36, 1651, 2033],
 [119, 368, 348, 3128, 1138, 3371, 2306, 851, 36, 1651, 2033],
 [4166, 1944, 2343, 1587, 1694, 293, 3939, 2499, 2306, 169],
 [4032, 2850, 1893, 3421, 4947, 3551, 4315, 1933],
 [1228, 693, 4087, 1585, 341, 4889, 1350, 2290, 2933, 3501, 2959],
 [3117, 1615, 1651, 293, 2306, 1138],
 [4981, 2912, 1025, 355, 4303, 4684, 553, 657, 4809],
 [4932, 2029, 2299, 1237, 4206, 3436, 4082, 36, 1651, 2033],
 [188, 4617, 4507, 4208, 1554, 36, 1651, 2033],
 [3815, 3656, 854, 3400, 3479, 4760, 1014, 3245, 230, 4326],
 [4011, 4707, 2542],
 [3788, 4537, 553, 4446, 2306, 4259, 3843, 3878],
 [2106, 322

In [37]:
sent_length=20
embedded_docs2=pad_sequences(onehot_repr2,padding='pre',maxlen=sent_length)
print(embedded_docs2)

[[   0    0    0 ... 4846 3526 1983]
 [   0    0    0 ... 4955 4095 3878]
 [   0    0    0 ... 3062  511 3211]
 ...
 [   0    0    0 ... 4951 2768 4550]
 [   0    0    0 ... 4951 2768 4550]
 [   0    0    0 ... 4951 2768 4550]]


In [38]:
embedded_docs2[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1584,
       2031, 2000, 3423, 1651, 3078, 3958, 4846, 3526, 1983], dtype=int32)

In [39]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
len(embedded_docs2),y.shape

(36570, (18285,))

In [41]:
import numpy as np
X_final=np.array(embedded_docs2)
y_final=np.array(y)

In [42]:
X_final.shape,y_final.shape

((36570, 20), (18285,))

In [43]:
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

In [44]:
from imblearn.under_sampling import RandomUnderSampler

In [45]:
# Reshape y_final to match the shape of X_final
y_final_reshaped = np.repeat(y_final, 2)

# Perform random undersampling to balance the dataset
rus = RandomUnderSampler(random_state=42)
X_final_balanced, y_final_balanced = rus.fit_resample(X_final, y_final_reshaped)

# Split the balanced data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_final_balanced, y_final_balanced, test_size=0.33, random_state=42)

# Verify the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (21236, 20)
y_train shape: (21236,)
X_test shape: (10460, 20)
y_test shape: (10460,)


In [48]:
y_final_reshaped.shape

(36570,)

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final_reshaped, test_size=0.33, random_state=42)

### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
383/383 [==============================] - 27s 64ms/step - loss: 0.6852 - accuracy: 0.5663 - val_loss: 0.6844 - val_accuracy: 0.5669
Epoch 2/10
383/383 [==============================] - 8s 20ms/step - loss: 0.6777 - accuracy: 0.5779 - val_loss: 0.6898 - val_accuracy: 0.5528
Epoch 3/10
383/383 [==============================] - 5s 13ms/step - loss: 0.6412 - accuracy: 0.6218 - val_loss: 0.7283 - val_accuracy: 0.5358
Epoch 4/10
383/383 [==============================] - 4s 11ms/step - loss: 0.5957 - accuracy: 0.6572 - val_loss: 0.7505 - val_accuracy: 0.5393
Epoch 5/10
383/383 [==============================] - 6s 15ms/step - loss: 0.5596 - accuracy: 0.6789 - val_loss: 0.8110 - val_accuracy: 0.5315
Epoch 6/10
383/383 [==============================] - 3s 8ms/step - loss: 0.5270 - accuracy: 0.6978 - val_loss: 0.9117 - val_accuracy: 0.5434
Epoch 7/10
383/383 [==============================] - 3s 7ms/step - loss: 0.4981 - accuracy: 0.7102 - val_loss: 0.9921 - val_accuracy: 0.5369


In [50]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Assuming you have already trained your model and obtained predictions on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)  # Convert predicted probabilities to binary labels

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


378/378 [==============================] - 1s 2ms/step
Confusion Matrix:
[[5227 1615]
 [4020 1207]]
Accuracy: 0.5331013339961885
